# Microbiome Relative Abundance Data Cleanup
## Anna Lapteva
### April 10, 2024

All the data used to generate the final .CSV document is sourced from [GMRepo](https://gmrepo.humangut.info/home) database. Specifically, all run IDs in the database linked to each project for each phenotype (Autism Spectrum Disorder (ASD), Alzheimer's Disease (AD), Parkinson's Disease (PD), Multiple Sclerosis (MS), and epilepsy) were saved as a spreadsheet, individually opened, and downloaded. Each downloaded .txt file is then cleaned and appended to a DataFrame in this code. The following projects, which perform 16S sequencing of fecal samples using Illumina tools, were used for data collection:
- ASD: PRJNA578223
- AD: PRJNA633959
- PD: PRJEB17784
- MS: PRJNA450340

Let's start by importing relevant packages.

In [1]:
# Relevant string operations
import string

# Data handling
import glob
import os
import pandas as pd

We will use the `glob` module, which allows us to get all our files of interest with the same extension for convenience.

In [2]:
# Specify directories holding data
data_dir = ['../Data/ASD', '../Data/AD', '../Data/PD', '../Data/MS']

# Initialize DataFrame for concatenation steps
df = pd.DataFrame()

# Iterate through each of the directories
for direc in data_dir:
    
    # Glob string for all .txt files, which have relative abundance data
    file_glob = os.path.join(direc, '*.txt')
    
    # Get list of files in direc
    file_list = glob.glob(file_glob)

    # Iterate through all files in list
    for file in file_list:

        # Initialize skip_row_id to be sufficiently large before the loop starts
        skip_row_id = len(file)

        # List to store modified lines
        modified_lines = []
        
        # Iterate through all lines in file
        with open(file, 'r') as f:
            # Find where relative abundance data starts
            for row_number, line in enumerate(f, start=1):
                if "ncbi_taxon_id" in line:
                    skip_row_id = row_number - 1
        
                # Ensure column entries are one "word"
                if row_number > skip_row_id:  # Ensure lines before are not edited
                    modified_line = ''
                    prev_char = None
                    for char in line:
                        if char == ' ' and prev_char in string.ascii_lowercase + '.':
                            modified_line += '_'
                        else:
                            modified_line += char
                        prev_char = char
                    modified_lines.append(modified_line)
        
        # Write modified lines back to file
        with open(file, 'w') as f:
            f.writelines(modified_lines)

        # Create DataFrame from file, removing filler info
        data = pd.read_csv(file, sep='\s+')

        # Add individual index for reshaping
        data["individual"] = 0

        # Reshape data
        data = data[["relative_abundance", "scientific_name", "individual"]].pivot(columns="scientific_name", values="relative_abundance", index="individual").reset_index()
        data = data.drop("individual", axis=1)

        # Add label
        data["Diagnosis"] = [direc[direc.rfind('/') + 1:]] * len(data)

        # Update DataFrame
        df = pd.concat([df, data], ignore_index=True).fillna(0)

# Check out df
df.head()

scientific_name,Alistipes,Bacteroides,Bifidobacterium,Butyricicoccus,Clostridium,Dorea,Eggerthella,Faecalibacterium,Gemmiger,Prevotella,...,Trichococcus,Anaerococcus,Caulobacter,Elizabethkingia,Flavobacterium,Methylobacterium,Burkholderia,Anoxybacillus,Epulopiscium,Robinsoniella
0,2.481730,9.75944,0.730816,1.720460,1.53776,0.685140,0.548112,3.593180,1.294150,47.2442,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.556370,10.32600,2.478230,1.428890,17.60440,0.580487,0.000000,2.433580,1.038180,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,50.02380,0.665589,1.179040,12.19930,0.817724,0.000000,0.950841,1.255110,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.206700,15.85570,0.531984,1.621900,6.20215,1.621900,0.402232,18.165300,0.000000,15.2978,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.650566,40.53110,3.564740,0.828803,9.01880,2.094290,0.000000,1.969520,0.516888,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Finally, let's export the data as a CSV!

In [3]:
# Export to CSV
df.to_csv("../Data/20440_cleaned_data.csv", index=False)